# Pneumonia Detection with Resnet50

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
image = ('Pneumonia')

In [12]:
# Count the number of images in Pneumonia and Normal folders
print('Total number of images in Pneumonia folder:', len(os.listdir('Pneumonia')))
print('Total number of images in Normal folder:', len(os.listdir('Pneumonia/NORMAL')))
print('Total number of images in Pneumonia folder:', len(os.listdir('Pneumonia/PNEUMONIA')))

Total number of images in Pneumonia folder: 2
Total number of images in Normal folder: 1342
Total number of images in Pneumonia folder: 3876


In [13]:
# Create a data generator for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create a training set
train_set = train_datagen.flow_from_directory('Pneumonia', target_size=(224, 224), batch_size=32, class_mode='binary')

# Create a validation set
val_set = val_datagen.flow_from_directory('Pneumonia', target_size=(224, 224), batch_size=32, class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [14]:
# Create a ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [16]:
# Freeze the layers of the model
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer
predictions = Dense(1, activation='sigmoid')(x)

# Create a model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Create a checkpoint
checkpoint = ModelCheckpoint('Pneumonia_Resnet50.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

# Create an early stopping
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')


In [17]:
# Train the model
history = model.fit(train_set, validation_data=val_set, epochs=50, callbacks=[checkpoint, early])

Epoch 1/50
 24/163 [===>..........................] - ETA: 15:36 - loss: 0.5868 - accuracy: 0.7383

In [ ]:
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


In [ ]:
# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')

In [ ]:
# Save the model
model.save('Pneumonia_Resnet50.h5')